In [2]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

In [4]:
df=pd.read_csv('flipkart_com-ecommerce_sample.csv',index_col=0)
df=df.dropna()
df=df.drop(['crawl_timestamp','product_url','overall_rating'],axis=1)
df=df.reset_index()

In [5]:
a=np.array(df['product_rating'])
a=[float(x) if x!="No rating available" else 4.1 for x in a]

In [6]:
df['product_rating_mediated']=a

In [7]:
category_tree=list(df['product_category_tree'])
category_tree=[x[2:-2] for x in category_tree]
split_category_tree=[x.split(' >>') for x in category_tree]
category=[]
for x in split_category_tree:
    category.append(x[0])
sub_category=[]
for x in split_category_tree:
    if(len(x)>1):
        sub_category.append(x[1])
    else:
        sub_category.append(x[0])


In [8]:
df['category']=category
df['sub_category']=sub_category

In [9]:
df

,uniq_id,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,brand,product_specifications,product_rating_mediated,category,sub_category
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Clothing,Women's Clothing
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",4.1,Furniture,Living Room Furniture
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",4.1,Footwear,Women's Footwear
3,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Clothing,Women's Clothing
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",4.1,Pet Supplies,Grooming
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14053,7179d2f6c4ad50a17d014ca1d2815156,WallDesign Small Vinyl Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7KFJAKSTDY9G,1500.0,730.0,"[""http://img6a.flixcart.com/image/wall-decorat...",False,Buy WallDesign Small Vinyl Sticker for Rs.730 ...,No rating available,WallDesign,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Baby Care,Baby & Kids Gifts
14054,71ac419198359d37b8fe5e3fffdfee09,Wallmantra Large Vinyl Stickers Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE9F5URNQGJCGH,1429.0,1143.0,"[""http://img6a.flixcart.com/image/sticker/z/g/...",False,Buy Wallmantra Large Vinyl Stickers Sticker fo...,No rating available,Wallmantra,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Baby Care,Baby & Kids Gifts
14055,93e9d343837400ce0d7980874ece471c,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7VAYDKQZEBSD,1299.0,999.0,"[""http://img5a.flixcart.com/image/sticker/b/s/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Baby Care,Baby & Kids Gifts
14056,669e79b8fa5d9ae020841c0c97d5e935,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE8YSVEPPCZ42Y,1499.0,1199.0,"[""http://img5a.flixcart.com/image/sticker/4/2/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Baby Care,Baby & Kids Gifts


In [10]:
def random_float_in_range(mean, std_dev, min_val, max_val):
    while True:
        value = np.random.normal(mean, std_dev)
        if min_val <= value <= max_val:
            return round(value,1)

min_range = 1
max_range = 5
mean_value = (min_range + max_range) / 2
std_deviation = (max_range - min_range) / 6  

random_float = random_float_in_range(mean_value, std_deviation, min_range, max_range)

In [11]:
import random
item_id=[]
ratings=[]
num_of_cust=1000
num_of_products=100
for x in range(num_of_cust):
    temp=[]
    temp2=[]
    for y in range(num_of_products):
        temp.append(random.randint(0,len(df)-1))
        temp2.append(random_float_in_range(mean_value, std_deviation, min_range, max_range))
    item_id.append(temp)
    ratings.append(temp2)

In [12]:
customers=pd.DataFrame()
customers['item_id']=item_id
customers['ratings']=ratings

In [13]:
customers

,item_id,ratings
0,"[8856, 6535, 7897, 10571, 11697, 8054, 13857, ...","[3.8, 3.0, 1.9, 2.1, 2.5, 3.4, 2.9, 3.1, 3.4, ..."
1,"[167, 11533, 12326, 5491, 10250, 11027, 2027, ...","[3.3, 3.2, 2.7, 3.0, 3.7, 3.1, 3.5, 3.4, 2.3, ..."
2,"[6265, 8135, 12307, 6024, 3365, 10226, 2896, 3...","[3.6, 3.6, 2.8, 1.5, 2.0, 3.3, 2.1, 3.9, 1.8, ..."
3,"[12199, 11212, 9299, 1031, 7800, 543, 6906, 23...","[3.2, 3.1, 2.3, 2.5, 2.1, 2.3, 3.1, 3.6, 3.5, ..."
4,"[4973, 13431, 4936, 8602, 8608, 6431, 736, 554...","[3.0, 2.0, 2.5, 3.5, 2.2, 3.1, 3.5, 2.8, 3.4, ..."
...,...,...
995,"[1779, 11893, 427, 9979, 13954, 13147, 1626, 9...","[3.2, 2.0, 2.6, 3.9, 4.2, 2.9, 3.4, 3.4, 2.0, ..."
996,"[5356, 2864, 8717, 6159, 10327, 10941, 9553, 2...","[2.4, 3.7, 2.5, 3.9, 2.4, 3.2, 3.2, 2.7, 2.7, ..."
997,"[9213, 10974, 6011, 3715, 9928, 4096, 10869, 1...","[2.4, 3.2, 3.9, 3.3, 3.3, 3.2, 4.7, 3.0, 3.2, ..."
998,"[11944, 3033, 5158, 615, 3037, 11201, 7136, 41...","[3.4, 2.9, 2.0, 3.0, 2.3, 1.8, 3.2, 3.2, 3.7, ..."


In [14]:
label_encoder = LabelEncoder()
encoded_categories = label_encoder.fit_transform(df['category'])
df['encoded_categories']=encoded_categories

In [15]:
num_rows = num_of_cust
num_columns = len(df['category'].unique())
data = [[0] * num_columns for _ in range(num_rows)]
columns = [i for i in range(num_columns)]

cust_encoded = pd.DataFrame(data, columns=columns)
cust_encoded

,0,1,2,3,4,5,6,7,8,9,...,251,252,253,254,255,256,257,258,259,260
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df.iloc[256]

uniq_id                                     9d9bd8488bf09dc468ecf58f7caeeaf0
product_name                        Karatcraft Trilic Gold Diamond 18 K Ring
product_category_tree                                 ["Jewellery >> Rings"]
pid                                                         RNGE88FRA9DMA4UZ
retail_price                                                         49999.0
discounted_price                                                     33761.0
image                      ["http://img5a.flixcart.com/image/ring/y/h/x/r...
is_FK_Advantage_product                                                False
description                Karatcraft Trilic Gold Diamond 18 K Ring\n    ...
product_rating                                           No rating available
brand                                                             Karatcraft
product_specifications     {"product_specification"=>[{"key"=>"Pack of", ...
product_rating_mediated                                                  4.1

In [17]:
for i in range(len(customers)):
    cust=customers.iloc[i]
    for j in range(len(cust['item_id'])):
        # print(i,j)
        # print(cust['item_id'][j])
        # print(df.iloc[cust['item_id'][j]]['encoded_categories'])
        col=df.iloc[cust['item_id'][j]]['encoded_categories']
        val=cust['ratings'][j]
        cust_encoded.iloc[i][col]+=val
        # print(cust_encoded.iloc[i][col])

print(cust_encoded.value_counts())

C:\Users\User\AppData\Local\Temp\ipykernel_24736\784931000.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  cust_encoded.iloc[i][col]+=val
C:\Users\User\AppData\Local\Temp\ipykernel_24736\784931000.py:9: FutureWarning: Setting an item of 

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  199  200  201  202  203  204  205  206  207  208  209  210  211  212  213  214  215  216  217  218  219  220  221  

In [18]:
num_clusters = int(len(cust_encoded)/10)
kmeans = KMeans(n_clusters=num_clusters)
cluster_labels = kmeans.fit_predict(cust_encoded)

# Add cluster labels to the DataFrame
cust_encoded['cluster_label']=cluster_labels
# cust_encoded

In [19]:
cust_encoded['cluster_label'].value_counts()

cluster_label
47    32
62    28
18    27
27    26
26    24
      ..
74     2
99     1
90     1
96     1
54     1
Name: count, Length: 100, dtype: int64

In [20]:
median_value = np.median(cluster_labels)

index_closest_to_median = np.argmin(np.abs(cluster_labels - median_value))

# Get the label (value) corresponding to the index
label_of_median = cluster_labels[index_closest_to_median]


In [21]:
label=np.bincount(cluster_labels).argmax()
group=cust_encoded[cust_encoded['cluster_label']==label]
target_customer=group.index[0]
group

,0,1,2,3,4,5,6,7,8,9,...,252,253,254,255,256,257,258,259,260,cluster_label
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
33,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
39,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
82,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47


In [22]:
len(df['sub_category'].unique())

442

In [23]:
label_encoder = LabelEncoder()
encoded_subcategories = label_encoder.fit_transform(df['sub_category'])
df['encoded_subcategories']=encoded_subcategories

In [24]:
expanded_group=pd.DataFrame()
num_rows = cust_encoded['cluster_label'].value_counts()[label]
num_columns = len(df['sub_category'].unique())
data = [[0] * num_columns for _ in range(num_rows)]
columns = [i for i in range(num_columns)]

expanded_group = pd.DataFrame(data, columns=columns)
expanded_group.set_index(pd.Index(group.index),inplace=True)


In [25]:
for i in range(len(expanded_group)):
    cust_index=expanded_group.index[i]
    cust=customers.iloc[cust_index]
    for j in range(len(cust['item_id'])):
        col=df.iloc[cust['item_id'][j]]['encoded_subcategories']
        val=cust['ratings'][j]
        expanded_group.iloc[i][col]+=val
expanded_group_t=(expanded_group.transpose())
expanded_group_t

C:\Users\User\AppData\Local\Temp\ipykernel_24736\3054200046.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  expanded_group.iloc[i][col]+=val
C:\Users\User\AppData\Local\Temp\ipykernel_24736\3054200046.py:7: FutureWarning: Setting an item

,28,33,39,82,88,101,122,196,231,254,...,668,698,726,776,811,830,852,886,922,937
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,3,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,4,0,0,3,0,0,4,0,...,3,3,0,3,4,3,0,3,3,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
row_sums = expanded_group_t.sum(axis=1)
expanded_group_t['sum_of_ratings']=row_sums
expanded_group_t=expanded_group_t[expanded_group_t['sum_of_ratings']!=0]
expanded_group_t

,28,33,39,82,88,101,122,196,231,254,...,698,726,776,811,830,852,886,922,937,sum_of_ratings
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,3,0,0,5,0,...,0,0,0,0,0,0,0,0,0,14
3,0,0,4,0,0,3,0,0,4,0,...,3,0,3,4,3,0,3,3,0,55
8,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,3
10,3,0,0,3,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,3,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4,0,4
353,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,0,4
431,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,2


In [27]:
expanded_group_t=expanded_group_t.drop('sum_of_ratings',axis=1)

In [28]:
num_clusters = int(len(expanded_group_t)/10)
kmeans = KMeans(n_clusters=num_clusters)
cluster_labels = kmeans.fit_predict(expanded_group_t)

# Add cluster labels to the DataFrame
expanded_group_t['cluster_label']=cluster_labels

In [29]:
expanded_group_t

,28,33,39,82,88,101,122,196,231,254,...,698,726,776,811,830,852,886,922,937,cluster_label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,0,0,0,0,0,3,0,0,5,0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,4,0,0,3,0,0,4,0,...,3,0,3,4,3,0,3,3,0,5
8,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,4
10,3,0,0,3,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4,0,4
353,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,0,0
431,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,4


In [30]:
expanded_group_t['cluster_label'].value_counts()

cluster_label
4    51
0     6
5     2
3     1
2     1
1     1
Name: count, dtype: int64

In [31]:
target_customer

28

In [32]:
ind=customers.iloc[target_customer]['item_id'][-1]
row=df.iloc[ind]['encoded_subcategories']

In [33]:
target_cluster_label=expanded_group_t.loc[row]['cluster_label']

In [34]:
req_cust=pd.DataFrame(expanded_group_t[expanded_group_t['cluster_label']==target_cluster_label])
req_cust.drop('cluster_label',axis=1,inplace=True)
req_cust

,28,33,39,82,88,101,122,196,231,254,...,668,698,726,776,811,830,852,886,922,937
136,8,0,3,7,0,7,3,7,3,3,...,2,4,7,3,0,7,10,3,12,10


In [35]:
row_sums = req_cust.sum(axis=1)
req_cust['sum_of_ratings']=row_sums
req_cust=req_cust[req_cust['sum_of_ratings']>2]
req_cust.drop('sum_of_ratings',axis=1,inplace=True)

In [36]:
num_clusters = int((len(req_cust)+10)/10)
kmeans = KMeans(n_clusters=num_clusters)
cluster_labels = kmeans.fit_predict(req_cust)

# Add cluster labels to the DataFrame
req_cust['cluster_label']=cluster_labels

In [37]:
req_cust['cluster_label'].value_counts()

cluster_label
0    1
Name: count, dtype: int64

In [38]:
ind=customers.iloc[target_customer]['item_id'][-1]
row=df.iloc[ind]['encoded_subcategories']
target_cluster_label=req_cust.loc[row]['cluster_label']
target_cluster_label

0

In [39]:
final_cust=pd.DataFrame(req_cust[req_cust['cluster_label']==target_cluster_label])
final_cust

,28,33,39,82,88,101,122,196,231,254,...,698,726,776,811,830,852,886,922,937,cluster_label
136,8,0,3,7,0,7,3,7,3,3,...,4,7,3,0,7,10,3,12,10,0


In [40]:
subcat_ind=final_cust.index
decoded_labels = label_encoder.inverse_transform(encoded_categories)
subcat=[]
for x in subcat_ind:
    subcat.append(decoded_labels[x])
print(subcat)

[' Decorative Lighting & Lamps']


In [41]:
row=customers.iloc[target_customer]['item_id'][-1]
df.iloc[row]['sub_category']

' Necklaces & Chains'